# Курсовой проект. Двухуровневая рекомендательная система по данным соревнования Retail Hero


### Основное

Дедлайн - 29 декабря 23:59
Целевая метрика precision@5
Бейзлайн решения - MainRecommender
Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)
Hints:

Сначала просто попробуйте разные параметры MainRecommender:

N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)
Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)
Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)
Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)
Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

---

### Применение в бизнесе

- 2-ух уровневая система применяется во многих компаниях
- Зачастую уровней > 2
- Идем от более простых эвристик/моделей к более сложным
- Фичи из моделей первого уровня (embeddings, biases из ALS) можно использовать в последующих моделях

Также решения на основе 2-ух уровневых рекомендаций заняли все топ-10 мест в соревновании X5 Retail hero. 

### Как отбирать кандидатов?

Вариантов множество. Тут нам поможет *MainRecommender*. Пока в нем реализованы далеко не все возможные способы генерации кандидатов

- Генерируем топ-k кандидатов
- Качество кандидатов измеряем через **recall@k**
- recall@k показывает какую долю из купленных товаров мы смогли выявить (рекомендовать) нашей моделью

----

Pipline:
1. Рекомендуем 50 кандидатов среди товаров классическими методами
2. Оцениваем recall@k нашу кандидатную выдачу (выдача моделями 1-го уровня)
3. Получаем user-item датасет по кандидатным рекомендациям
4. Для такого датасета проставляем target купил/не купил товар по истории взаимодействий
5. На этом датасете строим lightGBM, предсказывающий купит или не купит пользователь данный товар
6. Рекомендовано ознакомиться и попробовать Light AutoML

# Практическая часть

Код для src, utils кастомизировал, recommender2 содержит метод, возвращающий user&item embeddings. Они пригодятся для модели второго уровня.

In [ ]:
#!pip install implicit

In [ ]:
#Только для Colab
from google.colab import drive
drive.mount('/content/drive')
root = root = '/content/drive/My Drive/Colab Notebooks/rec_sys/data/'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


# Модель второго уровня
from lightgbm import LGBMClassifier

import pickle

In [2]:
#!pip install lightautoml

In [3]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.tasks.common_metric import mean_quantile_error

In [5]:
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
     sys.path.append(module_path)
#sys.path.append('/content/drive/My Drive/Colab Notebooks/rec_sys')

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders2 import MainRecommender

### Загружаем данные и разделяем их для валидации out of time: -9 недель -6 недель

In [6]:
data = pd.read_csv('../retail_train.csv')#(root+'retail_train.csv')
item_features = pd.read_csv('../product.csv')#(root+'product.csv')
user_features = pd.read_csv('../hh_demographic.csv')#(root+'hh_demographic.csv')

# column processing
#data.columns = [col.lower() for col in data.columns]
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [7]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


Модель первого уровня. MainRecommender

In [8]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [9]:
recommender

### Варианты, как получить кандидатов

Можно потом все эти варианты соединить в один

(!) Если модель рекомендует < N товаров, то рекомендации дополняются топ-популярными товарами до N

### Измеряем recall@k


In [10]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [11]:
result_lvl_1.shape

(2154, 2)

In [12]:
users_lvl_1 = pd.DataFrame(data_train_lvl_1['user_id'].unique(),columns = ['user_id'])
users_lvl_1.shape

(2422, 1)

In [19]:
%%time
K_num = 50
result_lvl_1['als_rec'] = users_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=K_num))
result_lvl_1['own_rec'] = users_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=K_num))
result_lvl_1['sim_items'] = users_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=K_num))
result_lvl_1['sim_users'] = users_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=K_num))

Wall time: 8min 51s


## Расчет recall для отбора модели первого уровня - модель для отбора кандидатов

In [26]:
def calculate_recall_k(data, K): #data - pandas df
    for column in data.columns[2:]:
        yield column, data.apply(lambda row: recall_at_k(row[column], row['actual'], k=K), axis=1).mean()

In [27]:
recall_results = pd.DataFrame(sorted(calculate_recall_k(result_lvl_1, 50), key=lambda x: x[1],reverse=True), columns = ['Candidate_model','Recall'])
recall_results

,Candidate_model,Recall
0,own_rec,0.083280
1,als_rec,0.067966
2,sim_items,0.026298
3,sim_users,0.006976


# Бейзлайн - модели первого уровня, расчет метрики precision@5

In [28]:
def calculate_precision_k(data, K): #data - pandas df
    for column in data.columns[2:]:
        yield column, data.apply(lambda row: precision_at_k(row[column], row['actual'], k=K), axis=1).mean()

In [29]:
precision_results = pd.DataFrame(sorted(calculate_precision_k(result_lvl_1, 5), key=lambda x: x[1],reverse=True), columns = ['Model','Precision'])
precision_results

,Model,Precision
0,own_rec,0.218199
1,als_rec,0.166760
2,sim_users,0.075580
3,sim_items,0.029898


### Лучшая метрика по Baseline - MainRecommender own (own рекомендации с взвешиванием tfidf, 10 факторов, доподненные ТОП популярными товарами) составляет 0.2182. Наилучший recall выдает метод own_recommendations (0.08328); его возьмем для отбора кандидатов второй модели. 

In [30]:
## Добавить ТОП популярных, стекнуть несколько кандидатских списков

### Обучаем модель 2-ого уровня на выбранных кандидатах

- Обучаем на data_train_lvl_2
- Обучаем *только* на выбранных кандидатах -  сгенерирую топ-50 кадидиатов через get_als_recommendations. Если юзер купил < 50 товаров, то get_als_recommendations дополнит рекоммендации топ-популярными

In [31]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [32]:
users_lvl_2.shape[0]

106600

In [33]:
users_lvl_2['user_id'].nunique()

2132

In [34]:
targets_lvl_2.shape

(112225, 3)

In [35]:
targets_lvl_2['target'].mean()

0.13711739808420584

## Feature generation. Добавим фичи users, items, а также их ембеддинги (ALS) из встроенного метода класса MainRecommender (добавил сам)

**Фичи user_id:**
    - Средний чек
    - Средняя сумма покупки 1 товара в каждой категории
    - Кол-во покупок в каждой категории
    - Частотность покупок раз/месяц
    - Долю покупок в выходные
    - Долю покупок утром/днем/вечером

**Фичи item_id**:
    - Кол-во покупок в неделю
    - Среднее ол-во покупок 1 товара в категории в неделю
    - (Кол-во покупок в неделю) / (Среднее кол-во покупок 1 товара в категории в неделю)
    - Цена (Можно посчитать из retil_train.csv)
    - Цена / Средняя цена товара в категории
    
**Фичи пары user_id - item_id**
    - (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)
    - (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
    - (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)

## Feature generation

In [36]:
#Max week
MAX_WEEK = data['week_no'].max()

In [37]:
# Данные транзакций
t_data = data_train_lvl_2.copy()
df_augm = targets_lvl_2
t_data = t_data.merge(item_features[['item_id','department']], on='item_id',how='left')

In [38]:
# средний чек на юзера
avg_basket = (t_data.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()).groupby('user_id')['sales_value'].mean().reset_index()
avg_basket.columns = ['user_id', 'avg_basket']

In [39]:
#Среднее кол-во покупок юзера в каждой категории
avg_user_qty_per_department = (t_data.groupby(['user_id', 'department'])['quantity'].sum().reset_index()).groupby('user_id')['quantity'].mean().reset_index()
avg_user_qty_per_department.columns = ['user_id', 'avg_user_qty_per_department']

In [40]:
# Количество недель после последней покупки юзера
last_activity = t_data.groupby(['user_id'])['week_no'].max().reset_index()
last_activity.columns = ['user_id', 'inactivity']
last_activity['inactivity'] = MAX_WEEK - last_activity['inactivity']

In [41]:
#цена товара
price = t_data.groupby(['item_id'])['sales_value','quantity'].sum().reset_index()
price['price'] = price['sales_value']/price['quantity']
price.drop(['sales_value','quantity'], axis=1,inplace=True)


<ipython-input-41-129824c5fbd6>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  price = t_data.groupby(['item_id'])['sales_value','quantity'].sum().reset_index()


In [42]:
# Среднее кол-во покупок 1 товара в категории
qty_purch_in_department = (t_data.groupby(['item_id', 'department'])['quantity'].sum().reset_index()).groupby('item_id')['quantity'].mean().reset_index()
qty_purch_in_department.columns = ['item_id', 'avg_count_item_dep']

In [43]:
items_emb = recommender.items_embedings()
users_emb = recommender.user_embedings()

In [44]:
df_augm = df_augm.merge(avg_basket, on='user_id',how='left')
df_augm = df_augm.merge(avg_user_qty_per_department, on='user_id',how='left')
df_augm = df_augm.merge(last_activity, on='user_id',how='left')
df_augm = df_augm.merge(user_features, on='user_id', how='left')
df_augm = df_augm.merge(users_emb, on='user_id', how='left')

In [45]:
df_augm = df_augm.merge(price[['item_id','price']], on='item_id',how='left')
df_augm = df_augm.merge(qty_purch_in_department, on='item_id',how='left')
df_augm = df_augm.merge(item_features, on='item_id', how='left')
df_augm = df_augm.merge(items_emb, on='item_id', how='left')

In [46]:
df_augm.shape

(112225, 41)

In [47]:
df_augm.head()

,user_id,item_id,target,avg_basket,avg_user_qty_per_department,inactivity,age_desc,marital_status_code,income_desc,homeowner_desc,...,item0,item1,item2,item3,item4,item5,item6,item7,item8,item9
0,2070,1082185,1.0,14.355581,1755.0,4,45-54,U,50-74K,Unknown,...,0.031869,0.061769,0.005470,-0.083484,0.064490,0.116423,0.046586,-0.095957,-0.082073,0.162250
1,2070,995242,1.0,14.355581,1755.0,4,45-54,U,50-74K,Unknown,...,0.063278,0.008416,0.021401,-0.082523,0.012280,0.121477,0.078272,-0.047995,-0.073631,0.126219
2,2070,1029743,0.0,14.355581,1755.0,4,45-54,U,50-74K,Unknown,...,0.004577,0.063225,0.008429,-0.044307,0.003955,0.109211,0.016099,-0.018862,-0.128738,0.179661
3,2070,1106523,0.0,14.355581,1755.0,4,45-54,U,50-74K,Unknown,...,0.019977,0.124507,0.029606,-0.030245,-0.016113,-0.030447,0.090976,-0.023069,-0.156031,0.145865
4,2070,1133018,0.0,14.355581,1755.0,4,45-54,U,50-74K,Unknown,...,0.065344,0.062236,0.042675,-0.074659,-0.003690,0.004270,0.156314,-0.034473,-0.109744,0.079326


In [73]:
targets_lvl_2 = df_augm

## Соединяю всю предобработку в функцию

In [76]:
def preprocessing(data):
    #Max week
    MAX_WEEK = data['week_no'].max()
    users_lvl_2 = pd.DataFrame(data['user_id'].unique())
    users_lvl_2.columns = ['user_id']

    train_users = data_train_lvl_1['user_id'].unique()
    users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

    users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
    s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'item_id'

    users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
    users_lvl_2['flag'] = 1
    targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
    targets_lvl_2['target'] = 1  # тут только покупки 

    targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

    targets_lvl_2['target'].fillna(0, inplace= True)
    targets_lvl_2.drop('flag', axis=1, inplace=True)
    #print(targets_lvl_2.shape)
    # feature augmenting and combining
    t_data = data.copy()
    df_augm = targets_lvl_2
    t_data = t_data.merge(item_features[['item_id','department']], on='item_id',how='left')
    # средний чек на юзера
    avg_basket = (t_data.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()).groupby('user_id')['sales_value'].mean().reset_index()
    avg_basket.columns = ['user_id', 'avg_basket']
    #Среднее кол-во покупок юзера в каждой категории
    avg_user_qty_per_department = (t_data.groupby(['user_id', 'department'])['quantity'].sum().reset_index()).groupby('user_id')['quantity'].mean().reset_index()
    avg_user_qty_per_department.columns = ['user_id', 'avg_user_qty_per_department']
    # Количество недель после последней покупки юзера
    last_activity = t_data.groupby(['user_id'])['week_no'].max().reset_index()
    last_activity.columns = ['user_id', 'inactivity']
    last_activity['inactivity'] = MAX_WEEK - last_activity['inactivity']
    #цена товара
    price = t_data.groupby(['item_id'])['sales_value','quantity'].sum().reset_index()
    price['price'] = price['sales_value']/price['quantity']
    price.drop(['sales_value','quantity'], axis=1,inplace=True)
    # Среднее кол-во покупок 1 товара в категории
    qty_purch_in_department = (t_data.groupby(['item_id', 'department'])['quantity'].sum().reset_index()).groupby('item_id')['quantity'].mean().reset_index()
    qty_purch_in_department.columns = ['item_id', 'avg_count_item_dep']

    items_emb = recommender.items_embedings()
    users_emb = recommender.user_embedings()

    df_augm = df_augm.merge(avg_basket, on='user_id',how='left')
    df_augm = df_augm.merge(avg_user_qty_per_department, on='user_id',how='left')
    df_augm = df_augm.merge(last_activity, on='user_id',how='left')
    df_augm = df_augm.merge(user_features, on='user_id', how='left')
    df_augm = df_augm.merge(users_emb, on='user_id', how='left')

    df_augm = df_augm.merge(price[['item_id','price']], on='item_id',how='left')
    df_augm = df_augm.merge(qty_purch_in_department, on='item_id',how='left')
    df_augm = df_augm.merge(item_features, on='item_id', how='left')
    df_augm = df_augm.merge(items_emb, on='item_id', how='left')

    return df_augm

In [50]:
cat_feats= ['age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc', 'hh_comp_desc',
       'household_size_desc', 'kid_category_desc','manufacturer',
       'department', 'brand', 'commodity_desc', 'sub_commodity_desc',
       'curr_size_of_product']

In [77]:
data_lvl_2 = preprocessing(data_train_lvl_2)
data_lvl_2.shape

(112225, 3)


<ipython-input-76-a9d178c0fcf4>:39: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  price = t_data.groupby(['item_id'])['sales_value','quantity'].sum().reset_index()


(112225, 41)

In [78]:
data_lvl_2[cat_feats] = data_lvl_2[cat_feats].astype('category')


In [79]:
%%time
data_test_2 = preprocessing(data_val_lvl_2)
data_test_2.shape

(106383, 3)


<ipython-input-76-a9d178c0fcf4>:39: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  price = t_data.groupby(['item_id'])['sales_value','quantity'].sum().reset_index()


Wall time: 8.85 s


(106383, 41)

In [80]:
data_test_2[cat_feats] = data_test_2[cat_feats].astype('category')

In [81]:
data_test_2.head(2)

,user_id,item_id,target,avg_basket,avg_user_qty_per_department,inactivity,age_desc,marital_status_code,income_desc,homeowner_desc,...,item0,item1,item2,item3,item4,item5,item6,item7,item8,item9
0,338,1082185,0.0,31.249333,17.777778,0,NaN,NaN,NaN,NaN,...,0.031869,0.061769,0.005470,-0.083484,0.06449,0.116423,0.046586,-0.095957,-0.082073,0.162250
1,338,995242,0.0,31.249333,17.777778,0,NaN,NaN,NaN,NaN,...,0.063278,0.008416,0.021401,-0.082523,0.01228,0.121477,0.078272,-0.047995,-0.073631,0.126219


In [82]:
data_test_2

,user_id,item_id,target,avg_basket,avg_user_qty_per_department,inactivity,age_desc,marital_status_code,income_desc,homeowner_desc,...,item0,item1,item2,item3,item4,item5,item6,item7,item8,item9
0,338,1082185,0.0,31.249333,17.777778,0,NaN,NaN,NaN,NaN,...,0.031869,0.061769,0.005470,-0.083484,0.064490,0.116423,0.046586,-0.095957,-0.082073,0.162250
1,338,995242,0.0,31.249333,17.777778,0,NaN,NaN,NaN,NaN,...,0.063278,0.008416,0.021401,-0.082523,0.012280,0.121477,0.078272,-0.047995,-0.073631,0.126219
2,338,1029743,0.0,31.249333,17.777778,0,NaN,NaN,NaN,NaN,...,0.004577,0.063225,0.008429,-0.044307,0.003955,0.109211,0.016099,-0.018862,-0.128738,0.179661
3,338,1106523,0.0,31.249333,17.777778,0,NaN,NaN,NaN,NaN,...,0.019977,0.124507,0.029606,-0.030245,-0.016113,-0.030447,0.090976,-0.023069,-0.156031,0.145865
4,338,1133018,1.0,31.249333,17.777778,0,NaN,NaN,NaN,NaN,...,0.065344,0.062236,0.042675,-0.074659,-0.003690,0.004270,0.156314,-0.034473,-0.109744,0.079326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106378,832,1004906,0.0,30.740000,5.333333,0,NaN,NaN,NaN,NaN,...,0.056264,0.071677,0.017306,-0.058145,0.069493,-0.021309,0.064047,0.083470,-0.008077,0.051269
106379,832,1023720,0.0,30.740000,5.333333,0,NaN,NaN,NaN,NaN,...,0.062860,0.107031,0.029776,0.002888,0.147866,0.021425,0.040160,-0.094336,0.033923,-0.069670
106380,832,5568378,0.0,30.740000,5.333333,0,NaN,NaN,NaN,NaN,...,0.039506,0.044023,-0.017386,0.075620,-0.085523,0.052324,-0.024307,0.074802,-0.017721,0.132000
106381,832,6034857,0.0,30.740000,5.333333,0,NaN,NaN,NaN,NaN,...,-0.042915,0.028498,-0.021189,-0.005060,0.120736,0.099418,0.023784,0.012300,0.062541,-0.021744


In [83]:
# Создание списка рекомендаций на основе вероятностей из predict_proba
def get_recomendations(test_data, test_preds, data_val_lvl_2):
    test_data['predict'] = test_preds

    test_data.sort_values(['user_id', 'predict'], ascending=False, inplace=True)

    result = test_data.groupby('user_id').head(5)

    recs = result.groupby('user_id')['item_id']
    recomendations = []
    for user, preds in recs:
        recomendations.append({'user_id': user, 'recomendations': preds.tolist()})

    recomendations = pd.DataFrame(recomendations)

    result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
    result_lvl_2.columns=['user_id', 'actual']

    result_lvl_2 = result_lvl_2.merge(recomendations)
    
    return result_lvl_2

In [84]:
data_lvl_2.columns

Index(['user_id', 'item_id', 'target', 'avg_basket',
       'avg_user_qty_per_department', 'inactivity', 'age_desc',
       'marital_status_code', 'income_desc', 'homeowner_desc', 'hh_comp_desc',
       'household_size_desc', 'kid_category_desc', 'user0', 'user1', 'user2',
       'user3', 'user4', 'user5', 'user6', 'user7', 'user8', 'user9', 'price',
       'avg_count_item_dep', 'manufacturer', 'department', 'brand',
       'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product', 'item0',
       'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'item7', 'item8',
       'item9'],
      dtype='object')

In [167]:
#data_lvl_2.to_csv('data_lvl_2.csv')

In [85]:
X_train = data_lvl_2.drop(['user_id', 'item_id','target'], axis=1)
y_train = data_lvl_2[['target']]

In [86]:
%%time
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

train_preds = lgb.predict(X_train)

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\lightgbm\basic.py:863: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Wall time: 2.45 s


In [87]:
train_preds

array([1., 1., 1., ..., 0., 0., 0.])

Берем топ-k предсказаний, ранжированных по вероятности, для каждого юзера

In [88]:
train_preds.mean()

0.05263533080864335

In [89]:
len(train_preds)

112225

In [90]:
X_train['preds'] = train_preds

In [91]:
X_train[['user_id', 'item_id','target']] = targets_lvl_2[['user_id', 'item_id','target']]


In [92]:
X_train[X_train.preds == 1]['user_id'].nunique()

1094

In [93]:
X_train['user_id'].nunique()

2132

In [94]:
X_test = data_lvl_2.drop(['target'], axis=1)

In [95]:
lgb_test_pred = lgb.predict_proba(X_test.drop(['user_id','item_id'],axis=1))[:,1]

In [96]:
X_test['predict'] = lgb.predict_proba(X_test.drop(['user_id','item_id'],axis=1))[:,1]
X_test['predict']

0         0.804189
1         0.574541
2         0.750031
3         0.585690
4         0.385261
            ...   
112220    0.019515
112221    0.001997
112222    0.034616
112223    0.008123
112224    0.022382
Name: predict, Length: 112225, dtype: float64

In [97]:
#Рекомендации после Lightgbm
result_test_1 = get_recomendations(X_test, lgb_test_pred, data_val_lvl_2)
result_test_1

,user_id,actual,recomendations
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 1082185, 1082185, 981760, 995242]"
1,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1082185, 1082185, 1029743, 1106523, 995242]"
2,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 1082185, 1029743, 995242, 1106523]"
3,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1082185, 1082185, 1029743, 1029743, 995242]"
4,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[1029743, 1082185, 1106523, 1126899, 995242]"
...,...,...,...
1899,2496,[6534178],"[1082185, 1082185, 1029743, 1106523, 1106523]"
1900,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[1082185, 1029743, 1106523, 883404, 981760]"
1901,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1082185, 1029743, 995242, 981760, 1070820]"
1902,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[1029743, 1082185, 1106523, 883404, 995242]"


In [98]:
result_test_1.apply(lambda row: precision_at_k(row['recomendations'], row['actual'], k=5), axis=1).mean()

0.1905462184873917

## Метрика СНИЗИЛАСЬ по сравнению с метрикой бейзлайна - рекоммендации get_own_recommendations (было 0,2182). Модель не тюнил, фокус на AutoML

LAMA как вторая модель

In [176]:
TASK = Task('reg', loss='rmsle', metric = mean_quantile_error, greater_is_better=False)
TIMEOUT = 400
N_THREADS = 4
MEMORY_LIMIT = 7
N_FOLDS = 5
RANDOM_STATE = 100
TARGET_NAME = 'target'
TEST_SIZE=0.2

In [177]:
roles = {'target': TARGET_NAME, 'drop': ['user_id', 'item_id']}

In [178]:
lama_model = TabularAutoML(task=TASK,
                            timeout=TIMEOUT,
                            cpu_limit = N_THREADS,
                            memory_limit = MEMORY_LIMIT,
                            gpu_ids='all',
                            reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                             
                            general_params={'use_algos': [ ['cb_tuned','lgb_tuned'], ['xgb'] ]},
                             
                            tuning_params={'max_tuning_iter': 10},
)

In [179]:
%%time
train_pred = lama_model.fit_predict(data_lvl_2, roles = roles)

INFO:optuna.storages._in_memory:A new study created in memory with name: no-name-f6075a44-73f8-407f-9b5b-2ff0b43153bb
INFO:optuna.study.study:Trial 0 finished with value: -0.06120467807587499 and parameters: {'feature_fraction': 0.6872700594236812, 'num_leaves': 244}. Best is trial 0 with value: -0.06120467807587499.
INFO:optuna.storages._in_memory:A new study created in memory with name: no-name-ee11c5a7-7ce7-426c-88f8-ea6f205f5def
Custom logger is already specified. Specify more than one logger at same time is not thread safe.INFO:optuna.study.study:Trial 0 finished with value: -0.06849906489503355 and parameters: {'max_depth': 4}. Best is trial 0 with value: -0.06849906489503355.
INFO:optuna.study.study:Trial 1 finished with value: -0.0648310265396962 and parameters: {'max_depth': 7}. Best is trial 1 with value: -0.0648310265396962.
INFO:optuna.study.study:Trial 2 finished with value: -0.06623978692926147 and parameters: {'max_depth': 6}. Best is trial 1 with value: -0.0648310265396

Wall time: 6min 52s


In [180]:
train_preds = lama_model.predict(targets_lvl_2.drop('target',axis=1))

In [181]:
train_preds

array([[0.63487667],
       [0.44008395],
       [0.5719971 ],
       ...,
       [0.02747615],
       [0.00796405],
       [0.04579499]], dtype=float32)

In [182]:
test_preds = lama_model.predict(data_test_2.drop('target',axis=1))


In [183]:
test_preds = test_preds.data

In [184]:
result_test_2 = get_recomendations(data_test_2, test_preds, data_val_lvl_2)
result_test_2

,user_id,actual,recomendations
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 1082185, 1082185, 995242, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1029743, 1082185, 951590, 1106523, 1053690]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1082185, 1082185, 1029743, 994928, 994928]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 1082185, 1029743, 1106523, 981760]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1082185, 1082185, 1029743, 1029743, 962568]"
...,...,...,...
2012,2496,[6534178],"[1082185, 1082185, 883404, 1029743, 1106523]"
2013,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[1082185, 1029743, 981760, 1106523, 995785]"
2014,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1082185, 1029743, 1106523, 1106523, 1070820]"
2015,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[1082185, 1029743, 883404, 1106523, 866211]"


In [185]:
result_test_2.apply(lambda row: precision_at_k(row['recomendations'], row['actual'], k=5), axis=1).mean()

0.19871095686663032

## Метрика незначительно улучшилась по сравнению с Lightgbm, однако хуже натюненного бейзлайна.
### Пробовал loss mse, rmsle, менял количество итераций. 
Лучшие гиперпараметры: loss rmsle, 
'use_algos': [ ['cb_tuned','lgb_tuned'], ['xgb'] ]
Локальная машина не позволила запустить полноценно.
Оставлены параметры с оптимальным результатом (таймаут маленьки).
Необходимо поработать в будущем с фичами, а также менять количество факторов для эмбеддингов.
Попробую еще максимально выжать из LAMA на Colab в будущем.
Сохраняю рекомендации натюненного бейзлайна (get_own_recommendations)

In [ ]:
# Сохраняем рекоммендации в файл.
#result_test_2.to_csv('recomendations_als.csv')

# Выводы
По сравнению с бейзлайном метрика значительно ухудшена. Целевой порог (0.18) превышен.
Лучшая метрика 0.218199
Извиняюсь за слабое оформление, фокус был на освоение LAMA, были нюансы, полезная, но нужны мощные ресурсы.

In [195]:
final_result = result_lvl_1[['user_id','actual','own_rec']].copy()
final_result['own_rec'] = final_result['own_rec'].apply(lambda x: x[:5])
final_result

,user_id,actual,own_rec
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[1082185, 995242, 1029743, 1106523, 1133018]"
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[1082185, 995242, 1029743, 1106523, 1133018]"
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67...","[1082185, 995242, 1029743, 1106523, 1133018]"
3,6,"[1024306, 1102949, 6548453, 835394, 940804, 96...","[1082185, 995242, 1029743, 1106523, 1133018]"
4,7,"[836281, 843306, 845294, 914190, 920456, 93886...","[1082185, 995242, 1029743, 1106523, 1133018]"
...,...,...,...
2149,2496,"[831509, 867188, 1013623, 1048851, 5592734, 16...","[1082185, 995242, 1029743, 1106523, 1133018]"
2150,2497,"[820291, 824759, 838797, 859010, 859075, 86077...","[1082185, 995242, 1029743, 1106523, 1133018]"
2151,2498,"[865511, 962991, 1076374, 1102358, 5564901, 15...","[1082185, 995242, 1029743, 1106523, 1133018]"
2152,2499,"[861282, 921744, 1050968, 13842089, 828837, 86...","[1082185, 995242, 1029743, 1106523, 1133018]"


In [196]:
final_result.to_csv('recomendations_als.csv')